In [1]:
import json
import requests
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib             
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from datetime import datetime 
from dingtalkchatbot.chatbot import DingtalkChatbot    #钉钉机器人
import smtplib
from email.header import Header
from email.mime.text import MIMEText
print(matplotlib.matplotlib_fname())                    #解决负号无法显示问题
# from matplotlib import font_manager 
requests.packages.urllib3.disable_warnings()
matplotlib.rc("font",family='Microsoft YaHei')           #设置中文字体
id = 1616731200
def time_switch(id):                                    #将时间戳转换为日期格式
    id = int(id)
    timeArray = time.localtime(id)
    otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)    #"%Y-%m-%d %H:%M:%S"
#    print(timeArray)
    return otherStyleTime
# time_switch(id)
# https://api.hbdm.com/linear-swap-ex/market/history/kline?

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\matplotlibrc


In [2]:
def huobi_u_plot(coinID, period, size, jc):                       #date为上一个次季到当季合约的切换日，格式为21****
    url_temp_qh = "https://api.hbdm.com/linear-swap-ex/market/history/kline?contract_code="+coinID+"-USDT&period="+period+"&size="+size    #合约价
    resp_temp_qh = requests.get(url_temp_qh, verify = False)
    df_temp_qh = pd.DataFrame(resp_temp_qh.json()['data'])          #获取期货数据
    url_temp_xh = "https://api.huobi.pro/market/history/kline?period="+period+"&size="+str(len(df_temp_qh))+"&symbol="+str.lower(coinID)+"usdt"  #现货价    
    resp_temp_xh = requests.get(url_temp_xh, verify = False)
    if resp_temp_xh.json()['status'] == 'error':                    #若没有该现货交易对，则取消本次计算，换下一个代币
        pass;
    else:
        df_temp_xh = pd.DataFrame(resp_temp_xh.json()['data'])
        df_qh = df_temp_qh[['id','close']]
        df_xh = df_temp_xh[['id','close']].sort_values('id', ascending=True).reset_index(drop = True)  #按照id正向排序并刷新索引
        if len(df_qh) != len(df_xh):
            pass;
        else:
            price_jc = df_qh['close'] - df_xh['close']  #求价差
            df_xh_price = []                            #定义现货价格列表
            for i in df_xh['close']:                    #提取现货价格输入到列表中
                df_xh_price.append(i)
            time_list = []                              #定义时间序列列表，格式为年月日
            time_list_tmp = []                          #定义时间序列列表，格式为TS（时间戳）
            for i in df_qh['id']:                      
                time_list_tmp.append(int(i))            #将时间戳输入到列表time_list_tmp中
                time = time_switch(i)                   #将时间戳调用函数转换为年月日格式
                time_list.append(time)                  #将年月日格式时间输入到列表中     
            jc_per_list = []
            jc_per_display= []
            for n in range(0,len(df_temp_qh)):
                jc_per_temp = price_jc[n]/df_xh['close'][n]                  # 求价差比值
                jc_per_list.append(jc_per_temp)                              #将价差比值输入到价差列表中
                warn =  pd.DataFrame()
                if max(jc_per_list) > float(jc):          #价差绝对值大于一定程度触发告警
                    warn = pd.DataFrame([[coinID,max(jc_per_list)]],columns=['币种','价差'])
                    return warn
                elif min(jc_per_list) < -float(jc):
                    warn = pd.DataFrame([[coinID,min(jc_per_list)]],columns=['币种','价差'])
                    return warn

In [3]:
def get_all_coin_u():                                              #获取所有币种函数
    all_coin_url = "https://api.hbdm.com/linear-swap-api/v1/swap_contract_info"
    req_all_temp_coin = requests.get(all_coin_url, verify = False)
    df_all_temp_coin = pd.DataFrame(req_all_temp_coin.json()['data'])['symbol']
    all_temp_coin = [i for i in df_all_temp_coin]
    return list(set(all_temp_coin))

In [4]:
def sendemail():
    mail_host = 'smtp.qq.com'  
    #163用户名
    mail_user = '909962673@qq.com'  
    #密码(部分邮箱为授权码) 
    mail_pass = 'aodmqkelbuafbdgf'   
    #邮件发送方邮箱地址
    sender = '909962673@qq.com'  
    #邮件接受方邮箱地址，注意需要[]包裹，这意味着你可以写多个邮件地址群发
    receivers = ['909962673@qq.com'] #['361694142@qq.com','7940175@qq.com', '476125228@qq.com']#,'909962673@qq.com'],#'476125228@qq.com']  
#     content = "卖一*和真实*差值为" +"\n" \

    message = MIMEText(content,'plain','utf-8')
    if abs(warnlist['价差'].max()) >= 0.1 or abs(warnlist['价差'].min()) >= 0.1:
        message['Subject'] = Header("会议跟进"+str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())), 'utf-8')
    elif abs(warnlist['价差'].max()) >= 0.075 or abs(warnlist['价差'].min()) >= 0.075:
        message['Subject'] = Header("会议纪要"+str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())), 'utf-8')      
    elif abs(warnlist['价差'].max()) >= 0.05 or abs(warnlist['价差'].min()) >= 0.05:
        message['Subject'] = Header("会议准备"+str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())), 'utf-8')   
    elif abs(warnlist['价差'].max()) >= 0.025 or abs(warnlist['价差'].min()) >= 0.025:
        message['Subject'] = Header("会议通知"+str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())), 'utf-8')   
    else:
        message['Subject'] = Header("无需关注"+str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())), 'utf-8')   
    message['From'] = sender 
    #接受方信息     
    message['To'] = receivers[0]
    #登录并发送邮件
    try:
        smtpObj = smtplib.SMTP() 
        #连接到服务器
        smtpObj.connect(mail_host,25)
        #登录到服务器
        smtpObj.login(mail_user,mail_pass) 
        #发送
        smtpObj.sendmail(
        sender,receivers,message.as_string()) 
        #退出
        smtpObj.quit() 
        print('success')
    except smtplib.SMTPException as e:
        print('error',e) #打印错误

In [5]:
def dingding():          #钉钉机器人通知程序
    webhook = 'https://oapi.dingtalk.com/robot/send?access_token=f0b9ff4b13e360145cf0ae9f4ff18aa957a9e7c02973fa17e88339262c65e62b'
    qianhai = DingtalkChatbot(webhook)
    if abs(warnlist['价差'].max()) >= 0.1 or abs(warnlist['价差'].min()) >= 0.1:
        tit = '会议跟进'
        text_tit = '会议跟进:\n'+\
        '> '
    elif abs(warnlist['价差'].max()) >= 0.075 or abs(warnlist['价差'].min()) >= 0.075:
        tit = '会议纪要'
        text_tit = '会议纪要:\n'
        '> '
    elif abs(warnlist['价差'].max()) >= 0.05 or abs(warnlist['价差'].min()) >= 0.05:
        tit = '会议准备'
        text_tit = '会议准备:\n'
        '> '
    elif abs(warnlist['价差'].max()) >= 0.025 or abs(warnlist['价差'].min()) >= 0.025:
        tit = '会议通知'
        text_tit = '会议通知:\n'
        '> '
    else:
        tit = '暂无会议'
        text_tit = '暂无会议:\n'
        '> ' 
    qianhai.send_markdown(title=tit, text=text_tit+content+str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())) ,is_at_all=False)


In [6]:
def sleeptime(hour,min,sec):
    return hour*3600 + min*60 + sec;
min_ = sleeptime(0,5,0);
second = sleeptime(0,0,1);

In [ ]:
count = 0
while 1 == 1:
    try:
        count += 1
        coinlist = []
        coinlist = get_all_coin_u()
        warnlist = pd.DataFrame()
        for i in coinlist:
            time.sleep(second)
            warn = huobi_u_plot(i, '1min', '5', '0.02')  #
            warnlist = warnlist.append(warn).reset_index(drop = True)
        print(count,end='  ')
        print(str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())),end='  ')
        print('%d 个币种' %len(coinlist))
        if len(warnlist) > 0:
            warnlist = warnlist.sort_values('价差', ascending=False).reset_index(drop = True)
        else:
            time.sleep(min_);
            continue;
        #     requests.packages.urllib3.disable_warnings()
        content = ''
        for i in warnlist.index:
        #     s = print('%s 币本位永续合约价差为 %s' %(testlist['币种'][i],'{:.1%}'.format(testlist['价差'][i])))
            s = warnlist['币种'][i]+' U本位永续合约价差为 '+'{:.1%}'.format(warnlist['价差'][i])+'\n'
            content += s
        if abs(warnlist['价差'].max()) >= 0.02 or abs(warnlist['价差'].min()) >= 0.02:
            sendemail()
            dingding()
        time.sleep(min_);
    except Exception as e:
        print(e)
        time.sleep(min_);
        continue;

1  2021-04-24 00:04:55  89 个币种
success
2  2021-04-24 00:11:37  89 个币种
success
3  2021-04-24 00:18:19  89 个币种
success
4  2021-04-24 00:25:01  89 个币种
success
5  2021-04-24 00:31:43  89 个币种
success
6  2021-04-24 00:38:25  89 个币种
success
7  2021-04-24 00:45:06  89 个币种
success
8  2021-04-24 00:51:47  89 个币种
success
9  2021-04-24 00:58:29  89 个币种
success
10  2021-04-24 01:05:10  89 个币种
success
11  2021-04-24 01:11:52  89 个币种
success
12  2021-04-24 01:18:33  89 个币种
success
13  2021-04-24 01:25:16  89 个币种
success
14  2021-04-24 01:31:58  89 个币种
success
15  2021-04-24 01:38:37  89 个币种
success
16  2021-04-24 01:45:16  89 个币种
success
17  2021-04-24 01:51:54  89 个币种
success
18  2021-04-24 01:58:33  89 个币种
success
19  2021-04-24 02:05:12  89 个币种
success
20  2021-04-24 02:11:50  89 个币种
success
21  2021-04-24 02:18:29  89 个币种
success
22  2021-04-24 02:25:08  89 个币种
success
23  2021-04-24 02:31:46  89 个币种
success
24  2021-04-24 02:38:25  89 个币种
success
25  2021-04-24 02:45:03  89 个币种
success
26  2021-